# License
    IPython notebook for simulating the heat equation with OpenCL
    Copyright (C) 2015 Andre.Brodtkorb@ifi.uio.no

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Lets have opencl ipython integration enabled
%load_ext pyopencl.ipython_ext

#Import packages we need
import numpy as np
import pyopencl as cl
import os
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

In [2]:
#Setup easier to use compilation of OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_CTX"] = "0"
os.environ["CUDA_CACHE_DISABLE"] = "1"

In [3]:
#Create OpenCL context
cl_ctx = cl.create_some_context()

#Create an OpenCL command queue
cl_queue = cl.CommandQueue(cl_ctx)

# Heat equation in 2D
The heat equation can be written
$$
\begin{align}
\frac{\partial u}{\partial t} &= \kappa \nabla^2 u\\
&=\kappa \left[ \frac{\partial^2 u}{\partial x^2} +  \frac{\partial^2 u}{\partial y^2} \right]
\end{align}
$$
where $u$ is the temperature, and $\kappa$ is the material specific heat conduction constant. 

By approximating the temporal derivative with a backward difference, and the spatial derivative with a central difference, we get
$$
\frac{1}{\Delta t} (u_{i, j}^{n+1} - u_{i, j}^{n}) 
= \kappa \left [
\frac{1}{\Delta x^2}(u_{i-1, j}^n - 2u_{i, j}^n + u_{i+1, j}^n)
+ \frac{1}{\Delta y^2}(u_{i, j-1}^n - 2u_{i, j}^n + u_{i, j+1}^n)
\right]
$$
and gathering $u^n+1$ on the left hand side and $u^n$on the right, we write
$$
u^{n+1}_{i,j} = u_{i,j}^n 
+ \frac{\kappa\Delta t}{\Delta x^2}(u_{i-1, j}^n - 2u_{i, j}^n + u_{i+1, j}^n)
+ \frac{\kappa\Delta t}{\Delta y^2}(u_{i, j-1}^n - 2u_{i, j}^n + u_{i, j+1}^n)
$$
This discretization is unstable if the following CFL condition is not met
$$
\frac{1}{2} \gt \frac{\kappa\Delta t}{\Delta x^2}, \qquad
\frac{1}{2} \gt \frac{\kappa\Delta t}{\Delta y^2}
$$
or 
$$
\Delta t \lt \text{min}\left(\frac{\Delta x^2}{2\kappa}, \frac{\Delta y^2}{2\kappa}\right)
$$

In [4]:
%%cl_kernel 

float3 F(const float3 Q, const float g) {
    float3 F;
    
    F.x= Q.y;
    F.y= (Q.y*Q.y)/Q.x + 0.5f*g*Q.x*Q.x;
    F.z= (Q.y*Q.z)/Q.x;
    
    return F;
}

float3 G(const float3 Q, const float g) {
    float3 G;
    
    G.x= Q.z;
    G.y= (Q.y*Q.z)/Q.x;
    G.z= (Q.z*Q.z)/Q.x + 0.5f*g*Q.x*Q.x;
    
    return G;
}

__kernel void SW_2D(__global float* Q1x, 
                    __global float* Q1y, 
                    __global float* Q1z,
                    __global const float* Q0x, 
                    __global const float* Q0y, 
                    __global const float* Q0z,
                    const float g, float dt, float dx, float dy) {
    //Get total number of cells
    int nx = get_global_size(0); 
    int ny = get_global_size(1);

    //Get position in grid
    int i = get_global_id(0); 
    int j = get_global_id(1); 
    int center = j*nx + i;
    int north = (j+1)*nx + i;
    int south = (j-1)*nx + i;
    int east = j*nx + i+1;
    int west = j*nx + i-1;

    //Internal cells
    if (i > 0 && i < nx-1 && j > 0 && j <ny-1) {
        float3 Q0north= (float3)(Q0x[north], Q0y[north], Q0z[north]);
        float3 Q0south= (float3)(Q0x[south], Q0y[south], Q0z[south]);
        float3 Q0east = (float3)(Q0x[east],  Q0y[east],  Q0z[east] );
        float3 Q0west = (float3)(Q0x[west],  Q0y[west],  Q0z[west] );
        
        
        float3 Q1 = 0.25f*(Q0east+ Q0west+ Q0north+ Q0south)
            - dt/(2.0f*dx)*(F(Q0east,  g) - F(Q0west,  g))
            - dt/(2.0f*dy)*(G(Q0north, g) - G(Q0south, g));
        
        Q1x[center]= Q1.x;
        Q1y[center]= Q1.y;
        Q1z[center]= Q1.z;
    }
}

In [5]:
%%cl_kernel
__kernel void SW_2D_bc(__global float* Q1x, __global float* Q1y, __global float* Q1z) {
    int nx = get_global_size(0); 
    int ny = get_global_size(1);
    int i = get_global_id(0); 
    int j = get_global_id(1);
    
    //Calculate the four indices of our neighboring cells
    int center = j*nx + i;
    int north = (j+1)*nx + i;
    int south = (j-1)*nx + i;
    int east = j*nx + i+1;
    int west = j*nx + i-1;
    
    if (i == 0) {
        Q1x[center]= Q1x[east];
        Q1y[center]= -Q1y[east];
        Q1z[center]= Q1z[east];
    }
    if (i == nx-1) {
        Q1x[center]= Q1x[west];
        Q1y[center]= -Q1y[west];
        Q1z[center]= Q1z[west];
    }
    if (j == 0) {
        Q1x[center]= Q1x[north];
        Q1y[center]= Q1y[north];
        Q1z[center]= -Q1z[north];
    }
    if (j == ny-1) {
        Q1x[center]= Q1x[south];
        Q1y[center]= Q1y[south];
        Q1z[center]= -Q1z[south];
    }
}

In [6]:
"""
Class that holds data for the heat equation in OpenCL
"""
class SWSim:
    """
    Uploads initial data to the CL device
    """
    def __init__(self, Q0x, Q0y, Q0z):
        #Make sure that the data is single precision floating point
        assert(np.issubdtype(Q0x.dtype, np.float32))
        assert(np.issubdtype(Q0y.dtype, np.float32))
        assert(np.issubdtype(Q0z.dtype, np.float32))
        assert(not np.isfortran(Q0x))
        assert(not np.isfortran(Q0y))
        assert(not np.isfortran(Q0z))
        
        assert(Q0x.shape == Q0y.shape)
        assert(Q0x.shape == Q0z.shape)
        
        #Find number of cells
        self.nx = Q0x.shape[1]
        self.ny = Q0x.shape[0]
        
        mf = cl.mem_flags 
        
        #Upload data to the device
        self.Q0x = cl.Buffer(cl_ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=Q0x)
        self.Q0y = cl.Buffer(cl_ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=Q0y)
        self.Q0z = cl.Buffer(cl_ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=Q0z)
        
        #Allocate output buffers
        self.Q1x = cl.Buffer(cl_ctx, mf.READ_WRITE, Q0x.nbytes)
        self.Q1y = cl.Buffer(cl_ctx, mf.READ_WRITE, Q0y.nbytes)
        self.Q1z = cl.Buffer(cl_ctx, mf.READ_WRITE, Q0z.nbytes)
        
    """
    Enables downloading data from CL device to Python
    """
    def downloadh(self): # return h = X
        #Allocate data on the host for result
        h = np.empty((self.ny, self.nx), dtype=np.float32)
        
        #Copy data from device to host
        cl.enqueue_copy(cl_queue, h, self.Q1x) 
        
        return h;
    
    def downloadhu(self): # return  u = Y/X
        #Allocate data on the host for result
        hu = np.empty((self.ny, self.nx), dtype=np.float32)
        
        #Copy data from device to host
        cl.enqueue_copy(cl_queue, hu, self.Q1y) 
        
        return hu;
    
    def downloadhv(self): # return v = Z/X
        #Allocate data on the host for result
        hv = np.empty((self.ny, self.nx), dtype=np.float32)
        
        #Copy data from device to host
        cl.enqueue_copy(cl_queue, hv, self.Q1z) 
        
        return hv;
    
    

In [7]:
"""
Computes the heat equation using an explicit finite difference scheme with OpenCL
"""
def opencl_SW_2D(cl_data, g, dx, dy, nt):
    #Calculate dt from the CFL condition
    dt = 0.05*min(dx, dy) #Estimate of dt that will not violate the CFL condition


    #Loop through all the timesteps
    for i in range(0, nt):
        #Execute program on device
        SW_2D(cl_queue, (cl_data.nx, cl_data.ny), None,
                       cl_data.Q1x, cl_data.Q1y, cl_data.Q1z,
                       cl_data.Q0x, cl_data.Q0y, cl_data.Q0z,
                       np.float32(g), np.float32(dt), np.float32(dx), np.float32(dy))
        
        #Impose boundary conditions
        SW_2D_bc(cl_queue, (cl_data.nx, cl_data.ny), None, 
                 cl_data.Q1x, cl_data.Q1y, cl_data.Q1z)
        
        #Swap variables
        cl_data.Q0x, cl_data.Q1x  = cl_data.Q1x, cl_data.Q0x
        cl_data.Q0y, cl_data.Q1y  = cl_data.Q1y, cl_data.Q0y
        cl_data.Q0z, cl_data.Q1z  = cl_data.Q1z, cl_data.Q0z

       

In [10]:
#Create test input data
nx=64
ny=128
Q0x = np.ones((ny,nx), dtype=np.float32)
Q0y = np.zeros((ny,nx), dtype=np.float32)
Q0z = Q0y


for j in range(ny):
    for i in range(nx):
        x=i-(nx/2.0)
        y=j-(ny/2.0)
        if(np.sqrt(x*x+y*y)<10):
            Q0x[j,i]=10.0

# np.set_printoptions(threshold=np.inf)
# print(Q0x)


In [11]:
cl_data = SWSim(Q0x, Q0y, Q0z)
g = 9.80665
dx = 100.0/float(nx)
dy = 100.0/float(ny)

#Plot initial conditions
h= Q0x;
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
fig.suptitle("SW 2D", fontsize=18)

max_x = cl_data.nx*dx
max_y = cl_data.ny*dy

y, x = np.mgrid[0:max_y:dy, 0:max_x:dx]
surf_args=dict(cmap=cm.coolwarm, shade=True, vmin=0.0, vmax=1.0, cstride=1, rstride=1)


def animate(i):
    timesteps_per_plot=10
    
    if (i>0):
        opencl_SW_2D(cl_data, g, dx, dy, timesteps_per_plot)
        
    h = cl_data.downloadh()
    
    ax.clear()
    ax.plot_surface(x, y, h, **surf_args)
    ax.set_zlim(-5.0, 5.0)



#opencl_SW_2D(cl_data, g, dx, dy, 2)

#h = cl_data.downloadh()
#hu = cl_data.downloadhu()
#hv = cl_data.downloadhv()

#np.set_printoptions(threshold=np.inf)
# print(h)
# print(hu)
# print(hv)


anim = animation.FuncAnimation(fig, animate, range(50), interval=100)
plt.close(anim._fig)
anim
